# [자연어처리]
# 5주차(5-1). 차원 축소(1)





# 202002961 김현주

# 이제 Colab 사용에 모든 수강생분들이 실행에는 어느정도 익숙해 졌을 거라 생각됩니다!
> ## 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해 보기 바랍니다. (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)





## PCA를 이용한 차원 축소



데이터 셋 준비 (우리에게 익숙한 20newsgroups)

In [1]:
from sklearn.datasets import fetch_20newsgroups

#20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

#학습 데이터셋을 가져옴
#메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
#검증 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

X_train = newsgroups_train.data   #학습 데이터셋
y_train = newsgroups_train.target #학습 데이터셋

X_test = newsgroups_test.data     #검증 데이터셋
y_test = newsgroups_test.target   #검증 데이터셋

전처리
* 토큰화, 불용어처리, 스테밍


In [2]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

RegTok = RegexpTokenizer("[\w']{3,}") # 정규포현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english')) #영어 불용어를 가져옴
stemmer = PorterStemmer() # 스테머로 포터스테머 사용

def my_tokenizer(text):
    tokens = RegTok.tokenize(text)
    # stopwords 제외
    words = [word for word in tokens if word not in english_stops]
    # portr stemmer 적용
    features = [stemmer.stem(token) for token in words]
    return features

In [4]:
# 약간의 시간(약 30초)이 소요됩니다.
from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf = TfidfVectorizer(tokenizer=my_tokenizer, max_features=2000, min_df=5, max_df=0.5)
# 토큰화 외에 아무것도 적용하지 않은 채로 확인
tfidf = TfidfVectorizer(tokenizer=my_tokenizer) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

X_train_tfidf = tfidf.fit_transform(X_train) # train set을 변환
X_test_tfidf = tfidf.transform(X_test) # test set을 변환

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:
print('Original tfidf matrix shape:', X_train_tfidf.shape)

Original tfidf matrix shape: (2034, 20085)


(방법 1) 차원 축소 방법을 적용하기 전의 Logistic Regression을 이용한 분류 성능 확인

In [6]:
from sklearn.linear_model import LogisticRegression

LR_clf = LogisticRegression() #분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test)))

#Train set score: 0.962
#Test set score: 0.761


(방법 2) 차원 축소 방법을 적용한 후의 Logistic Regression을 이용한 분류 성능 확인
* 차원 축소 방법인 PCA를 적용
* 2000차원으로 축소한 뒤 Logistic Regression을 이용한 분류 성능 확인

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

explained_variance_ratio_
- Percentage of variance explained by each of the selected components.
- If n_components is not set then all components are stored and the sum of the ratios is equal to 1.0.
- 각각의 새로운 축에 대해, 축소 전 분산에 대한 비율
- 만일 새로운 축들이 원래의 분산을 모두 설명해준다면 explained_variance_ratio_의 합은 1이 될 것임

In [7]:
# 약간의 시간(약 30초)이 소요됩니다.
from sklearn.decomposition import PCA

#  20,085 차원을 2,000 차원으로 축소
pca = PCA(n_components=2000, random_state=7) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

print('Original tfidf matrix shape:', X_train_tfidf.shape)
print('PCA Converted matrix shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum())) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

Original tfidf matrix shape: (2034, 20085)
PCA Converted matrix shape: (2034, 2000)
Sum of explained variance ratio: 1.000


In [8]:
# PCA로 차원 축소된 X 데이터 이용
LR_clf.fit(X_train_pca, y_train) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train))) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test))) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

#Train set score: 0.962
#Test set score: 0.760


(방법 3) 특성 선택 방식인 라쏘 회귀를 이용
* lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1)
* 차원을 축소

In [9]:
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1)
lasso_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('#Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))

import numpy as np
# 계수(coefficient) 중에서 0이 아닌 것들의 개수를 출력
print('#Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

#Train set score: 0.790
#Test set score: 0.718
#Used features count: 321 out of 20085


(방법 4) 특성 추출 추출 방식인 PCA를 이용
- 차원의 수를 322개로 축소
- pca = PCA(n_components=322, random_state=7)


In [10]:
# 약간의 시간(약 30초)이 소요됩니다.
pca = PCA(n_components=322, random_state=7) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())

# 322차원으로 줄어든 것을 확인
print('PCA Converted X shape:', X_train_pca.shape) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))  ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)
# 2,000개에 비해 정보량이 절반 이하로 줄어들었음

LR_clf.fit(X_train_pca, y_train)  ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.) # 322차원으로 줄어든 것을 확인
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))  ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test))) ### ... 이 부분을 완성하시오 (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)
# 라쏘 회귀분석(0.718)을 이용한 특성 선택보다 더 좋은 성능을 보임

PCA Converted X shape: (2034, 322)
Sum of explained variance ratio: 0.438
#Train set score: 0.871
#Test set score: 0.746


(방법 5) 그러면 322개보다 더 줄여서 100개만 이용해보자.
- 여전히 라쏘 회귀분석보다 좋은지 확인


In [11]:
# 약간의 시간(약 30초)이 소요됩니다.
pca = PCA(n_components=100, random_state=7)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())
X_test_pca = pca.transform(X_test_tfidf.toarray())
print('PCA Converted X shape:', X_train_pca.shape)
print('Sum of explained variance ratio: {:.3f}'.format(pca.explained_variance_ratio_.sum()))

LR_clf.fit(X_train_pca, y_train)
print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_pca, y_train)))
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_pca, y_test)))
# 라쏘 회귀분석(0.718)을 이용한 특성 선택보다 더 좋은 성능을 보임

PCA Converted X shape: (2034, 100)
Sum of explained variance ratio: 0.211
#Train set score: 0.808
#Test set score: 0.738


---

# **[[실습 5-1-1]]** 방법 1 ~ 방법 5의 Test set score를 비교하고, 어떤 방법이 가장 좋은지 결론을 내리시오. 그리고 그 이유를 추측해 보시오.

- 방법 1의 Test set score: 0.761
- 방법 2의 Test set score: 0.760
- 방법 3의 Test set score: 0.718
- 방법 4의 Test set score: 0.746
- 방법 5의 Test set score: 0.738


비교 결과 작성:
> 0.761 (방법1) > 0.760 (방법2) > 0.746 (방법4) > 0.738 (방법5) > 0.718 (방법3)

따라서 방법1이 가장 좋다. 이러한 차이가 발생한 이유는 차원 축소를 적용하지 않은 방법 1이 가장 큰 차원을 그대로 활용했기 때문이다.

차원 축소를 적용한 방법들을 비교하면, PCA를 이용한 방법2, 방법4, 방법5의 성능이 라쏘 회귀를 이용한 방법3보다 더 좋은 성능을 보였다. 또한 PCA를 이용한 방법은 차원의 크기를 라쏘 회귀보다 작게 했음에도 좋은 성능을 보였다. 이는 PCA는 데이터의 분산을 최대로 보존하는 새로운 축을 찾아 변환하여 차원을 축소하는데 이때 정보가 최대한 유지되도록하기 때문이다.

---

# [파일] -> [다운로드] -> [.ipynb 다운로드]